# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [44]:
import pandas as pd
import requests as r
import numpy as np

# 1) Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. 
# Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. 
# Há linhas duplicadas?
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [47]:
# 2) Conte o número de valores missing por variável.
na_count = sinasc.isna().sum()
na_count

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [48]:
# 3) Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem 
# ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:  
# ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']  
# Refaça a contagem de valores missings.
sinasc_filtrado = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_filtrado.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [49]:
# 4) Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente 
# à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. 
# Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número 
# de missings.

sinasc_apgar5 = sinasc_filtrado.dropna(subset=['APGAR5'])
sinasc_apgar5.isna().sum()



LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [50]:
# 5) observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o 
# não preenchido é o mesmo que o código 9.
sinasc_apgar5.loc[sinasc_apgar5['ESTCIVMAE'] == 9.0, 'ESTCIVMAE'] = np.nan
sinasc_apgar5.loc[sinasc_apgar5['CONSULTAS'] == 9, 'CONSULTAS'] = np.nan

sinasc_apgar5



,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0
2,1,37,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4.0,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4.0,9.0


In [51]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.

sinasc_apgar5.loc[sinasc_apgar5['QTDFILVIVO'].isna(), 'QTDFILVIVO'] = 0
sinasc_apgar5


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0
2,1,37,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4.0,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4.0,9.0


In [70]:
# 7) Das restantes, decida que valores te parecem mais adequados (um 'não preenchido' ou um valor 'mais provável' como no 
# item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista e que estamos tomando 
# decisões a todo o momento - não há necessariamente certo e errado aqui.


display(sinasc_apgar5['ESCMAE'].value_counts()) # por conta do número muito maior do mais frequente, entendo que o ideal seja substituir pelo valor mais frequente/provável
sinasc_apgar5.loc[sinasc_apgar5['ESCMAE'].isna(), 'ESCMAE'] = sinasc_apgar5['ESCMAE'].value_counts().idxmax()
display(sinasc_apgar5['GESTACAO'].value_counts()) # pelo mesmo motivo, essa coluna também será substituída da mesma forma
sinasc_apgar5.loc[sinasc_apgar5['GESTACAO'].isna(), 'GESTACAO'] = sinasc_apgar5['GESTACAO'].value_counts().idxmax()
display(sinasc_apgar5['GRAVIDEZ'].value_counts()) # essa coluna também segue a mesma lógica
sinasc_apgar5.loc[sinasc_apgar5['GRAVIDEZ'].isna(), 'GRAVIDEZ'] = sinasc_apgar5['GRAVIDEZ'].value_counts().idxmax()
sinasc_apgar5.isna().sum() # somente as colunas onde trocamos os valores ignorados por NaN continuam com valores NaN


ESCMAE
8 a 11 anos        15891
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: count, dtype: int64

GESTACAO
37 a 41 semanas        23739
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: count, dtype: int64

GRAVIDEZ
Única              26432
Dupla                488
Tríplice e mais        5
Name: count, dtype: int64

LOCNASC         0
IDADEMAE        0
ESTCIVMAE     459
ESCMAE          0
QTDFILVIVO      0
GESTACAO        0
GRAVIDEZ        0
CONSULTAS      50
APGAR5          0
dtype: int64

In [76]:
# 8) O Apgar possui uma classificação indicando se o bebê passou por asfixia:
'''
Entre 8 e 10 está em uma faixa 'normal'.
Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
Entre 4 e 5 significa 'asfixia moderada'.
Entre 0 e 3 significa 'asfixia severa'.
'''
# Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

sinasc_apgar5.loc[sinasc_apgar5['APGAR5'] >= 8, 'apgar_cat'] = 'normal'
sinasc_apgar5.loc[sinasc_apgar5['APGAR5'] < 8, 'apgar_cat'] = 'asfixia leve'
sinasc_apgar5.loc[sinasc_apgar5['APGAR5'] < 6, 'apgar_cat'] = 'asfixia moderada'
sinasc_apgar5.loc[sinasc_apgar5['APGAR5'] < 4, 'apgar_cat'] = 'asfixia severa'
sinasc_apgar5['apgar_cat'].value_counts()

apgar_cat
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [79]:
# 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palavras. 
# Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.
nomes_originais = list(sinasc_apgar5.columns)
novos_nomes = ['loc_nasc', 
               'idade_mae', 
               'est_civ_mae',
               'esc_mae', 
               'qtd_fil_vivo',
               'gestacao', 
               'gravidez', 
               'consultas',
               'apgar5', 
               'apgar5_cat']
novos_nomes = dict(zip(nomes_originais, novos_nomes))
novos_nomes

{'LOCNASC': 'loc_nasc',
 'IDADEMAE': 'idade_mae',
 'ESTCIVMAE': 'est_civ_mae',
 'ESCMAE': 'esc_mae',
 'QTDFILVIVO': 'qtd_fil_vivo',
 'GESTACAO': 'gestacao',
 'GRAVIDEZ': 'gravidez',
 'CONSULTAS': 'consultas',
 'APGAR5': 'apgar5',
 'apgar_cat': 'apgar5_cat'}